In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import mat73 as mt
import matplotlib.pyplot as plt

from fppnpx2.NPXSession import *

## Testing

In [3]:
cipath = "/home/mateoumaguing/Documents/MiscData/TibsCFD/TIBERIUS_CHKDLAY_DLPFC_NPIX45_063023_g0/kilosort4/cluster_info.tsv"

In [94]:
ci = pd.read_csv(cipath, sep="\t")

In [ ]:
ci = pd.read_csv(cipath, sep="\t")
ci = ci[ci['group'] == 'good']

In [21]:
ci.head()

,cluster_id,Amplitude,ContamPct,KSLabel,amp,ch,depth,fr,group,n_spikes,sh
1,1,22.1,4.3,good,46.004280,2,40.0,0.562448,good,3923,0.0
3,3,9.9,4.9,good,34.914692,0,20.0,0.475565,good,3317,0.0
4,4,10.8,0.0,good,38.051731,3,40.0,0.427249,good,2980,0.0
14,14,20.1,0.0,good,34.306488,6,80.0,2.188430,good,15264,0.0
16,16,11.8,50.8,mua,27.166042,8,100.0,1.013927,good,7072,0.0


In [8]:
wfpath = "/home/mateoumaguing/Documents/MiscData/TibsCFD/20230630_DLPFCwaveforms.mat"

In [46]:
agg = np.load("/home/mateoumaguing/Documents/MiscData/TibsCFD/TIBERIUS_CHKDLAY_DLPFC_NPIX45_063023_g0/kilosort4/templates.npy")

In [48]:
agg.shape

(551, 61, 383)

In [49]:
template_amps = np.max(np.abs(agg), axis=1)

In [50]:
template_amps.shape

(551, 383)

In [56]:
np.where(template_amps[0] > 0)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),)

In [98]:
np.where(template_amps[:,13] > 0)[0]

array([ 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 27])

In [12]:
wf = mt.loadmat(wfpath)['goodUnits']
wf_pd= pd.DataFrame(wf)

In [47]:
wf_pd.iloc[0:2]

,Amplitude,amp,channelId,cluId,cluster_waveforms,cluster_waveforms_time_relative,depth,fr,n_spikes,st,waveformTimeRelative,waveforms
0,25.4,50.22740173,15.0,17.0,"[[219.62215, nan], [228.13664, nan], [234.9485...","[-29, -28, -27, -26, -25, -24, -23, -22, -21, ...",160.0,5.35551717,37354.0,"[5033, 26765, 70175, 106570, 121640, 126151, 1...","[-20.0, 41.0]","[[6, 0, 4, -5, -4, 2, 4, -4, 0, 2, 10, -4, -2,..."
1,21.6,54.74397278,13.0,21.0,"[[92.525246, nan], [90.372826, nan], [89.44674...","[-29, -28, -27, -26, -25, -24, -23, -22, -21, ...",140.0,4.1603683,29018.0,"[1796, 4074, 7332, 21257, 32415, 35589, 43363,...","[-20.0, 41.0]","[[3, -2, 1, -1, 4, 9, -2, 3, 6, 1, 0, 6, 3, 2,..."


In [56]:
channel_map = np.load("/home/mateoumaguing/Documents/MiscData/TibsCFD/TIBERIUS_CHKDLAY_DLPFC_NPIX45_063023_g0/kilosort4/channel_map.npy")

In [87]:
spk_template_path = "/home/mateoumaguing/Documents/MiscData/TibsCFD/TIBERIUS_CHKDLAY_DLPFC_NPIX45_063023_g0/kilosort4/spike_templates.npy"

spk_templates = np.load(spk_template_path)
spk_templates.shape

(15907387,)

In [92]:
spk_clusters_path = "/home/mateoumaguing/Documents/MiscData/TibsCFD/TIBERIUS_CHKDLAY_DLPFC_NPIX45_063023_g0/kilosort4/spike_clusters.npy"

spk_clusters = np.load(spk_clusters_path)
spk_clusters.shape

(15907387,)

In [93]:
np.unique(spk_clusters)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 123, 124, 125, 126, 127, 128, 129, 130, 131,
       132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144,
       145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157,
       158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170,
       171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 18

## New stuff

In [3]:
imecpath = Path('/home/mateoumaguing/Documents/MiscData/TibsCFD/TIBERIUS_CHKDLAY_DLPFC_NPIX45_063023_g0')

In [4]:
tester = read_session([5,10], imec_path=imecpath, bandtype='AP')

Creating session from /home/mateoumaguing/Documents/MiscData/TibsCFD/TIBERIUS_CHKDLAY_DLPFC_NPIX45_063023_g0
Reading metadata...
Reading recorded signal...
383 usable channels. Use `channel_map` to get usable channels.
Reading cluster information from kilosort...
Session loading complete.


In [5]:
tester.keys()

dict_keys(['bandtype', 'meta', 'sampling_rate', 'N_channels', 'channel_map', 'N_channels_inuse', 'time_range', 'channel_ts_array', 'cluster_info', 'clusters_on_channels', 'cluster_spike_times', 'channels_for_clusters'])

In [16]:
waveforms = sio.loadmat("/home/mateoumaguing/Documents/MiscData/TibsCFD/CHKDLAY_063023_waveforms.mat")

In [44]:
testtt = waveforms["waveform_sample"].mean(axis=2)

In [59]:
waveforms['cluster_id']

array([[ 17,  21,  22,  24,  35,  41,  48,  52,  64,  65,  80,  87,  91,
         95, 105, 116, 117, 123, 125, 129, 130, 135, 139, 140, 146, 150,
        153, 177, 213, 215, 227, 234, 239, 241, 247, 248, 249, 250, 251,
        255, 257, 260, 263, 265, 268, 269, 272, 274, 275, 278, 284, 286,
        287, 291, 292, 296, 298, 306, 310, 311, 313, 314, 315, 317, 322,
        323, 328, 329, 332, 333, 335, 336, 339, 340, 344, 346, 347, 348,
        350, 354, 356, 360, 362, 365, 374, 375, 378, 379, 380, 383, 389,
        392, 393, 396, 397, 398, 401, 403, 405, 410, 417, 418, 419, 423,
        429, 433, 435, 439, 441, 443, 447, 449, 469, 470, 473, 480, 491,
        496, 500, 502, 516, 519, 523, 526, 528, 530, 531, 539, 548, 552,
        554, 556, 557, 561, 572, 583, 585, 587]])

In [ ]:
np.array([tester['cluster_info']['ch'][tester['cluster_info']['cluster_id'] == wci] for wci in waveforms['cluster_id'].squeeze()])

AttributeError: 'Series' object has no attribute 'value'

In [69]:
print(tester['cluster_info']['ch'][tester['cluster_info']['cluster_id'] == 17])

17    15
Name: ch, dtype: int64


In [58]:
tester['channels_for_clusters']

{1: array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 3: array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 4: array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 14: array([ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15]),
 16: array([ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),
 17: array([ 8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]),
 20: array([ 8,  9, 10, 11, 12, 13, 14, 15, 16, 17]),
 21: array([ 8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]),
 22: array([ 8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]),
 24: array([12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]),
 26: array([14, 15, 16, 17, 18, 19, 20, 21, 22, 23]),
 31: array([16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]),
 33: array([16, 17, 18, 19, 20, 21, 22, 23, 24, 25]),
 35: array([16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]),
 36: array([16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]),
 39: array([20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]),
 41: array([20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]),
 42: array([20, 21, 22, 23, 24, 25,